In [301]:
import random
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from numpy import linalg as LA

def alphagen(mu,sigma, S):
    Alpha = np.random.normal(mu/S, sigma/np.sqrt(S), size=(S, S))
    np.fill_diagonal(Alpha, 0)
    return Alpha

def antagonistic_matrix(S2, c2, mu2, sigma2):
    G2 = nx.erdos_renyi_graph(S2, c2/S2)
    adj_matrixG2 = nx.to_numpy_array(G2)
    weights = np.random.normal(mu2/c2 , sigma2/(np.sqrt(c2)),size=(S2,S2))
    np.fill_diagonal(weights, 0)
    Antagonistic_Matrix = adj_matrixG2*weights
    for i in range(S2):
        for j in range(i + 1, S2):  
            if Antagonistic_Matrix[i, j]*Antagonistic_Matrix[j, i] > 0:
                if np.random.rand() > 0.5:
                    Antagonistic_Matrix[i, j] = -Antagonistic_Matrix[i, j]
                else:
                    Antagonistic_Matrix[j, i] = -Antagonistic_Matrix[j, i]
    return Antagonistic_Matrix
    
def Mixture_matrix(S,mu,sigma,c):
    G = nx.erdos_renyi_graph(S, c/S)
    adj_matrix = nx.to_numpy_array(G)
    alpha = np.random.normal(mu/c , sigma/(np.sqrt(c)),size=(S,S))
    interaction_matirx = adj_matrix * alpha
    np.fill_diagonal(interaction_matirx, 0)
    return interaction_matirx

def get_mean_var(A):
    lst = np.ravel(A)
    other = []
    count = 0
    S = len(A[0])
    for i in lst:
        if i == 0:
            count = count + 1
        else:
            other.append(i)
    ot = np.array(other)
    mean = sum(ot)/(len(ot))
    print("Mean ", mean)
    var_sqrd = ot*ot
    moment2 = sum(var_sqrd)/(len(ot))
    var = moment2 - mean**2
    print("Var:", var)
    print("Num Vars: ", len(ot))
    return mean , var

def gamma_antagonistic(Tmu,Tsigma,Tc):
    Tm = Tmu/c
    Tsig = (Tsigma)/(np.sqrt(c))
    I = norm.cdf(-Tm/Tsig)
    f = norm.pdf(-Tm/Tsig)
    TCorr = -(Tm - 2*(Tm*I - Tsig*f))**2
    one = (Tsig-2)*Tm*Tsig*f +(Tsig**2 + Tm**2)*I
    TVar = 2*(Tsig**2 + Tm**2 - one)
    print(TVar)
    TGamma = TCorr/(TVar)
    return TGamma
    
def cal_gamma(cAlpha):
    cval = get_mean_var(cAlpha)
    cmu = cval[0]
    cVar = cval[1]
    count = 0
    clst = []
    cS = len(cAlpha[0])
    for i in range(cS):
        for j in range(i + 1, cS): 
            if cAlpha[i,j] == 0:
                count = count + 1
            else:
                centry = cAlpha[i,j]*cAlpha[j,i]
                clst.append(centry)
    ccorr = sum(clst)/(len(clst))
    ca = ((ccorr - (cmu**2))/cVar)
    return ca
   

In [297]:
S = 2000
mu = 1
sigma = 1
c = 1000

Alpha = antagonistic_matrix(S, c, mu, sigma)


In [290]:
b = gamma_antagonistic(mu,sigma,c)
print(b)

0.0010758922874543912
-0.5923051086552578


In [303]:
d = cal_gamma(Alpha)
print(d)

Mean  0.0002063239945898876
Var: 0.0010015635268321433
Num Vars:  1999850
-0.6366765187288594
